# Life Cycle Assessment

BioSTEAM facilitates life cycle assessment through a minimalistic interface with stream and utility objects. For demonstration purposes only, a simplified life cycle assessment is performed here for a sugarcane biorefinery assumming that materials other than sugarcane have negligible impacts. Our goal is to get the green house gas emission per kg ethanol along a cradle-to-gate system boundary.   

Assume the following inventory:

|             | GWP 100yr | Reference                                              |
| ----------- | --------- | ------------------------------------------------------ |
| Sugarcane   | 0.044     | https://www.osti.gov/servlets/purl/1337146             |
| Electricity | 0.480     | [https://greet.es.anl.gov/](https://greet.es.anl.gov/) |


In [22]:
import biosteam as bst
from biorefineries import sugarcane as sc
sc.load()
sys = sc.sugarcane_sys
feeds = sys.feeds
material_cradle_to_gate_key = ('GWP100', 'cradle_to_gate') 
electricity_key = ('GWP100', 'cradle_to_gate')  
for feed in sys.feeds:
    # Assume all other feeds are negligible
    feed.characterization_factors[material_cradle_to_gate_key] = 0.

# Set non-negligible characterization factors
sc.sugarcane.characterization_factors[material_cradle_to_gate_key] = 0.044
bst.PowerUtility.characterization_factors[electricity_key] = 0.48

# Get net electricity data
net_electricity = bst.PowerUtility.sum([i.power_utility for i in sys.units])
GWP_material = sum([s.get_impact(material_cradle_to_gate_key) for s in sys.feeds])
GWP_direct_emissions = sum([i.imass['CO2'] for i in sys.products if 'CO2' in i.chemicals])
GWP_electricity_production = net_electricity.get_impact(production_key=electricity_key)
GWP_total = GWP_material + GWP_direct_emissions + GWP_electricity_production # kg CO2 eq. / hr 
GWP_sugarcane_ethanol = GWP_total / sc.ethanol.F_mass # kg CO2 eq. / kg sugarcane ethanol
print(GWP_sugarcane_ethanol)

9.010823701688784
